In [ ]:
import matplotlib.pyplot as plt
import glob
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel

RootDir = '/mnt/research/turbulence/ml/data/'
fig, p = plt.subplots(2,3,figsize=(15,10),sharex=True)

B0_all = []
SolWeight_all = []
ForcingAmp_all = []
U_all = []
T_all = []

ke_all = []
me_all = []
sm_all = []
am_all = []
pb_all = []

dynTime_all = []

data_all = []

for Dir in glob.glob(RootDir + '256_*'):
    Dir_parts = Dir.split('_')
    B0 = float(Dir_parts[-1])
    SolenoidalWeight = float(Dir_parts[-2])
    ForcingAmpl = float(Dir_parts[-3])
    

    
    B0_all.append(B0)
    SolWeight_all.append(SolenoidalWeight)
    ForcingAmp_all.append(ForcingAmpl)
    

    try:
        data = np.genfromtxt(Dir + '/id0/Turb.hst')
        data_all.append(data)
    except:
        print('check ', Dir)
        continue
    
    
    # Estimating the rms velocity U based on the forcing amplitude.
    # This assumes a constant forcing efficiency (and is estimated based on solenoidal forcing)
    U = 0.5 * np.sqrt(ForcingAmpl) 
    
    U_all.append(U)
    
    # estimate dynamical/turnover time by characteristic length (here, 0.5 which is half the box size) and velocity
    T = 0.5 / U
    
    T_all.append(T)
    # convert code time to dynamical times
    data[:,0] /= T
    
    dynTime_all.append(data[:,0])
    
    Label = r'$B_0$ = %.2f; $\zeta = %.2f$; $a = %.2f$' % (B0,SolenoidalWeight,ForcingAmpl)
    
    p[0,0].set_ylabel('Kinetic energy')
    p[0,0].plot(data[:,0],data[:,14],label=Label)
    p[0,0].set_yscale('log')
    p[0,0].set_ylim(1e-4,3e0)
    
    ke_all.append(data[:,14])
    
    p[0,1].set_ylabel('Magnetic energy')
    p[0,1].plot(data[:,0],data[:,15],label=Label)
    p[0,1].set_yscale('log')
    p[0,1].set_ylim(5e-5,1e0)
    
    me_all.append(data[:,15])

    p[0,2].set_ylabel('Kinetic/Magnetic energy')
    p[0,2].plot(data[:,0],data[:,14]/data[:,15],label=Label)    
    p[0,2].set_yscale('log')
    p[0,2].set_ylim(1e0,3e2)

    p[1,0].set_ylabel('Sonic Mach')
    p[1,0].plot(data[:,0],data[:,16],label=Label)   
    
    sm_all.append(data[:,16])
    
    p[1,1].set_ylabel('Alfvenic Mach')
    p[1,1].plot(data[:,0],data[:,17],label=Label)  
    
    am_all.append(data[:,17])

    p[1,2].set_ylabel('Plasma beta')
    p[1,2].plot(data[:,0],data[:,19],label=Label)   
    p[1,2].set_yscale('log')
    
    pb_all.append(data[:,19])
    
fig.tight_layout()
p[0,0].legend()



In [ ]:
#Create input_parameter array
input_param = []
for i in range (len(B0_all)):
    {
        input_param.append([B0_all[i],SolWeight_all[i],ForcingAmp_all[i]])
    }

#Remove input parameters from simulation that did not run
input_param = input_param[0:7]+input_param[8:]

In [ ]:
all_data = [dynTime_all, ke_all, me_all, sm_all, am_all, pb_all]

#Create complete sample data
samples = [[],[],[],[],[],[]]

#through each data type(time, ke, me, etc)
for k in range(len(all_data)):
    #through data in each type
    for i in range(len(all_data[k])):
        sample = []
        #create sample linspace(10 pts)
        sample_lin = np.linspace(0, len(all_data[k][i])-1, 20)
        #print(sample_lin)
        for j in sample_lin:
            sample.append(all_data[k][i][int(j)])
            #print(j)
            #print(sample)
        samples[k].append(np.array(sample))
        #print(samples[k])

        
#Everything below can (should) be rewritten into a cleaner loop
dynTime_samples = samples[0]
ke_samples = samples[1]
me_samples = samples[2]
sm_samples = samples[3]
am_samples = samples[4]
pb_samples = samples[5]

#create training samples
tr_dT = dynTime_samples[:][:-1]
tr_in = input_param[:][:-1]
tr_ke = ke_samples[:][:-1]
tr_me = me_samples[:][:-1]
tr_sm = sm_samples[:][:-1]
tr_am = am_samples[:][:-1]
tr_pb = pb_samples[:][:-1]

#create test sample
te_dT = dynTime_samples[:][-1]
te_in = input_param[:][-1]
te_ke = ke_samples[:][-1]
te_me = me_samples[:][-1]
te_sm = sm_samples[:][-1]
te_am = am_samples[:][-1]
te_pb = pb_samples[:][-1]

In [ ]:
#Kinetic Energy Model

kern = RBF()+ConstantKernel()+WhiteKernel()
n_res = 10


gp_ke = GaussianProcessRegressor(kernel = kern, n_restarts_optimizer = n_res)
gp_ke.fit(tr_in, tr_ke)
ke_pred, ke_cov = gp_ke.predict(np.array(te_in).reshape(1,-1), return_cov = True)

for i in range(len(tr_ke)):

        plt.plot(tr_dT[i], tr_ke[i],alpha = 0.2)
        
plt.scatter(te_dT, te_ke, label = "Actual")
plt.scatter(te_dT, ke_pred[0], label = "Predicted")

plt.yscale('log')
plt.ylim(1e-4,3e0)
plt.legend()
#print(ke_pred)

#mean and std. dev. of predicted stationary data
stat_ke_pred = ke_pred[0][-5:]
mean_ke_pred = np.mean(stat_ke_pred)
std_ke_pred = np.std(stat_ke_pred)

#mean and std. dev. of actual stationary data (testing model)
stat_ke_act = te_ke[-5:]
mean_ke_act = np.mean(stat_ke_act)
std_ke_act = np.std(stat_ke_act)

print("Predicted Mean: %f, Actual Mean: %f" %(mean_ke_pred, mean_ke_act))
print("Predicted Standard Deviation: %f, Actual Standard Deviation: %f" %(std_ke_pred, std_ke_act))

In [ ]:
#Magnetic Energy Model

kern = RBF()+ConstantKernel()+WhiteKernel()
n_res = 10


gp_me = GaussianProcessRegressor(kernel = kern, n_restarts_optimizer = n_res)
gp_me.fit(tr_in, tr_me)
me_pred, me_cov = gp_me.predict(np.array(te_in).reshape(1,-1), return_cov = True)

for i in range(len(tr_me)):

        plt.plot(tr_dT[i], tr_me[i],alpha = 0.2)
        
plt.scatter(te_dT, te_me, label = "Actual")
plt.scatter(te_dT, me_pred[0], label = "Predicted")

plt.yscale('log')
plt.ylim(5e-5,1e0)
plt.legend()
#print(me_pred)

#mean and std. dev. of predicted stationary data
stat_me_pred = me_pred[0][-5:]
mean_me_pred = np.mean(stat_me_pred)
std_me_pred = np.std(stat_me_pred)

#mean and std. dev. of actual stationary data (testing model)
stat_me_act = te_me[-5:]
mean_me_act = np.mean(stat_me_act)
std_me_act = np.std(stat_me_act)

print("Predicted Mean: %f, Actual Mean: %f" %(mean_me_pred, mean_me_act))
print("Predicted Standard Deviation: %f, Actual Standard Deviation: %f" %(std_me_pred, std_me_act))

In [ ]:
#Sonic Mach Model

kern = RBF()+ConstantKernel()+WhiteKernel()
n_res = 10


gp_sm = GaussianProcessRegressor(kernel = kern, n_restarts_optimizer = n_res)
gp_sm.fit(tr_in, tr_sm)
sm_pred, sm_cov = gp_sm.predict(np.array(te_in).reshape(1,-1), return_cov = True)

for i in range(len(tr_sm)):

        plt.plot(tr_dT[i], tr_sm[i],alpha = 0.2)
        
plt.scatter(te_dT, te_sm, label = "Actual")
plt.scatter(te_dT, sm_pred[0], label = "Predicted")

plt.yscale('log')
plt.legend()
#print(sm_pred)

#mean and std. dev. of predicted stationary data
stat_sm_pred = sm_pred[0][-5:]
mean_sm_pred = np.mean(stat_sm_pred)
std_sm_pred = np.std(stat_sm_pred)

#mean and std. dev. of actual stationary data (testing model)
stat_sm_act = te_sm[-5:]
mean_sm_act = np.mean(stat_sm_act)
std_sm_act = np.std(stat_sm_act)

print("Predicted Mean: %f, Actual Mean: %f" %(mean_sm_pred, mean_sm_act))
print("Predicted Standard Deviation: %f, Actual Standard Deviation: %f" %(std_sm_pred, std_sm_act))

In [ ]:
#Alfvenic Mach Model

kern = RBF()+ConstantKernel()+WhiteKernel()
n_res = 10


gp_am = GaussianProcessRegressor(kernel = kern, n_restarts_optimizer = n_res)
gp_am.fit(tr_in, tr_am)
am_pred, am_cov = gp_am.predict(np.array(te_in).reshape(1,-1), return_cov = True)

for i in range(len(tr_am)):

        plt.plot(tr_dT[i], tr_am[i],alpha = 0.2)
        
plt.scatter(te_dT, te_am, label = "Actual")
plt.scatter(te_dT, am_pred[0], label = "Predicted")

plt.yscale('log')
plt.legend()
#print(am_pred)

#mean and std. dev. of predicted stationary data
stat_am_pred = am_pred[0][-5:]
mean_am_pred = np.mean(stat_am_pred)
std_am_pred = np.std(stat_am_pred)

#mean and std. dev. of actual stationary data (testing model)
stat_am_act = te_am[-5:]
mean_am_act = np.mean(stat_am_act)
std_am_act = np.std(stat_am_act)

print("Predicted Mean: %f, Actual Mean: %f" %(mean_am_pred, mean_am_act))
print("Predicted Standard Deviation: %f, Actual Standard Deviation: %f" %(std_am_pred, std_am_act))

In [ ]:
#Plasma Beta Model

kern = RBF()+ConstantKernel()+WhiteKernel()
n_res = 10


gp_pb = GaussianProcessRegressor(kernel = kern, n_restarts_optimizer = n_res)
gp_pb.fit(tr_in, tr_pb)
pb_pred, pb_cov = gp_pb.predict(np.array(te_in).reshape(1,-1), return_cov = True)

for i in range(len(tr_pb)):

        plt.plot(tr_dT[i], tr_pb[i],alpha = 0.2)
        
plt.scatter(te_dT, te_pb, label = "Actual")
plt.scatter(te_dT, pb_pred[0], label = "Predicted")

plt.yscale('log')
plt.legend()
#print(pb_pred)

#mean and std. dev. of predicted stationary data
stat_pb_pred = pb_pred[0][-5:]
mean_pb_pred = np.mean(stat_pb_pred)
std_pb_pred = np.std(stat_pb_pred)

#mean and std. dev. of actual stationary data (testing model)
stat_pb_act = te_pb[-5:]
mean_pb_act = np.mean(stat_pb_act)
std_pb_act = np.std(stat_pb_act)

print("Predicted Mean: %f, Actual Mean: %f" %(mean_pb_pred, mean_pb_act))
print("Predicted Standard Deviation: %f, Actual Standard Deviation: %f" %(std_pb_pred, std_pb_act))